In [1]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from collections import Counter
from pickle import dump

import sys
sys.path.append('../src')
from utils import w_label_prop
from quality_functions import eval_functions
from parse_data import parse_amazon

def read_graph(path="../datasets/fb_dg_gabr"):
    graph = nx.Graph()
    for i in range(N_NODES):
        graph.add_node(i)
    for v, line in enumerate(open(path)):
        edges = map(int, line.split())
        for u in edges:
            graph.add_edge(v, u)
    return graph

def get_comm_sizes_samples(graph, n_samples=2, verbose=True):
    ans = []
    deltas = []
    for i in tqdm(range(n_samples)):
        labels, cur_deltas = w_label_prop(graph)
        deltas.append(cur_deltas)
        comm_sizes = list(Counter(labels).values())
        ans.append(comm_sizes)
        if i < 5 and verbose:
            print(len(comm_sizes))
            top_sizes = sorted(comm_sizes)[::-1][:min(len(comm_sizes, 5))]
            print(top_sizes)
    return ans, deltas

graph_types = [
    'eps',
    'knn', 
    'inf'
]

f = open('../tmp_files/amazon_cd', "wb")

In [2]:
from time import time

all_deltas = []

for fname in ['conn'] + graph_types:
    t_begin = time()
    print(fname.upper())
    if fname == 'conn':
        graph = parse_amazon()
    else:
        graph = read_graph('../datasets/amazon_%s.graph' % fname)
    N_NODES = graph.number_of_nodes()
    print("N_nodes:", graph.number_of_nodes())
    print("N_edges:", graph.number_of_edges())
    print("Ratio:", graph.number_of_edges() / graph.number_of_nodes())
    labels, deltas = w_label_prop(graph)
    all_deltas.append(deltas)
    dump((labels, graph), f)
    del labels
    del graph
    print('time', time() - t_begin)
    
"""
KNN
N_nodes: 548553
N_edges: 2542747
Ratio: 4.63537160493152
time 451.4027636051178
"""
f.close()

297it [00:00, 2842.94it/s]

CONN


15010574it [00:51, 289038.73it/s]
100%|██████████| 548552/548552 [00:19<00:00, 28712.28it/s]


N_nodes: 548552
N_edges: 987942
Ratio: 1.8009997229068528
conn 229.16251301765442
EPS
N_nodes: 548552
N_edges: 3892707
Ratio: 7.096331797167816
eps 448.1566860675812
INF
N_nodes: 548553
N_edges: 1547577
Ratio: 2.821198680893186
inf 313.81835436820984


In [15]:
from pickle import load

from networkx.algorithms.components import connected_components

def comp_sizes(graph):
    comps = []
    for c in connected_components(graph):
        comps.append(len(c))
    return comps

def top_k(array, k=10):
    return sorted(array)[::-1][: min(k, len(array))]


all_graph_types = ['conn', 'eps', 'knn', 'inf']
cur_ind = 0
with open("../tmp_files/amazon_cd", "rb") as f_in:
    with open("../tmp_files/amazon_cd.pickle", "wb") as f_out:
        while True:
            try:
                t_begin = time()
                labels, graph = load(f_in)
                print(all_graph_types[cur_ind].upper())
                cur_ind += 1
                comm_sizes = list(Counter(labels).values())
                cmp_sizes = comp_sizes(graph)
                print("comm", top_k(comm_sizes))
                print("comp", top_k(cmp_sizes))
                dump((graph, comm_sizes, cmp_sizes), f_out)
                del graph
                del comm_sizes, cmp_sizes
                print(time() - t_begin)
            except BaseException as ex:
                print(ex)
                break

conn
comm [913, 813, 579, 522, 516, 433, 429, 422, 404, 353]
comp [334859, 222, 184, 131, 101, 90, 86, 79, 78, 75]
13.94341492652893
eps
comm [72687, 21382, 11910, 8076, 7341, 2993, 1924, 1754, 1715, 1406]
comp [155470, 101308, 25602, 12155, 3686, 3347, 2215, 924, 855, 756]
36.500447273254395
knn
comm [731, 716, 505, 455, 443, 428, 410, 388, 379, 361]
comp [369356, 95834, 43104, 5946, 61, 52, 48, 44, 26, 24]
20.1483051776886
inf
comm [193, 188, 175, 174, 168, 159, 159, 155, 154, 152]
comp [548364, 41, 40, 35, 26, 24, 13, 10]
27.393975734710693
Ran out of input
